### Import Necessary Libraries

In [123]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

### Data Preprocessing

In [124]:
IMG_SIZE = 128 
IMG_DIR = 'img'

data = []
labels = []

# Dictionary to map class labels to integers
label_map = {chr(i): i - ord('A') for i in range(ord('A'), ord('Y') + 1)}
if 'J' in label_map: # skip J and Z
    del label_map['J']

# Load images and process them
for class_label in os.listdir(IMG_DIR):
    class_dir = os.path.join(IMG_DIR, class_label)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            
            # Skip non-image files (like .DS_Store)
            if not img_name.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                print(f"Skipping non-image file: {img_path}")
                continue
            
            # Read image and check if it was read successfully
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue
            
            # Resize image
            img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            
            # Normalize pixel values
            img_normalized = img_resized / 255.0
            
            # Append data and labels
            data.append(img_normalized)
            labels.append(label_map[class_label])  # Map class_label to integer

data = np.asarray(data)
labels = np.asarray(labels)

Skipping non-image file: img/I/.DS_Store


In [125]:
# Reshape data to add channel dimension
data = data.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

# One-hot encode labels
labels = to_categorical(labels)
num_classes = labels.shape[1]

data.shape

(500, 128, 128, 1)

In [126]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)


### Model Training

In [127]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D((2, 2),padding='same'),
    Conv2D(64, (3, 3), activation='relu',padding='same'),
    MaxPooling2D((2, 2),padding='same'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [128]:
early_stopping = EarlyStopping(monitor='loss', 
                              min_delta=0.001,
                              patience= 5,
                              restore_best_weights= True, 
                              verbose = 0)

reduce_learning_rate = ReduceLROnPlateau(monitor='accuracy', 
                                         patience = 2, 
                                         factor=0.5 , 
                                         verbose = 1)

In [129]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [130]:
# Train the CNN model
history = model.fit(x_train, y_train, 
                    epochs=20, 
                    batch_size=32, 
                    callbacks=[early_stopping, reduce_learning_rate],
                    verbose=1)



# Save the model
model.save('model.keras')


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.0469 - loss: 3.4109 - learning_rate: 0.0010
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.1955 - loss: 2.9850 - learning_rate: 0.0010
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - accuracy: 0.5053 - loss: 2.3202 - learning_rate: 0.0010
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.6924 - loss: 1.3586 - learning_rate: 0.0010
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.8824 - loss: 0.6632 - learning_rate: 0.0010
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 134ms/step - accuracy: 0.9228 - loss: 0.3521 - learning_rate: 0.0010
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - accuracy: 0.9542 - loss: 0.1728 - learning_rate: 0.0010
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.9658 - loss: 0.1594 - learning_rate: 0.0010
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 0.9786 - loss: 0.0948 - learning_rate: 0.0010
Epoch 10/2

### Model Evaluation

In [131]:
# Model Evaluation on training data
train_loss, train_acc= model.evaluate(x_train, y_train)
print('Accuracy of the model for training data is:', train_acc * 100)
print('Loss of the model for training data is:', train_loss)

# Model Evaluation on testing data
test_loss, test_acc = model.evaluate(x_test,y_test)
print('Accuracy of the model for testing data is:',test_acc*100)
print('Loss of the model for testing data is:',test_loss)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 1.0000 - loss: 1.0957e-04
Accuracy of the model for training data is: 100.0
Loss of the model for training data is: 0.00012647711264435202
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9939 - loss: 0.0149   
Accuracy of the model for testing data is: 99.00000095367432
Loss of the model for testing data is: 0.024258196353912354


In [132]:
# Prediction
result = model.predict(x_test)

# Convert predictions and true labels from one-hot encoding to class labels
y_pred = np.argmax(result, axis=1)
y_true = np.argmax(y_test, axis=1)

# Find correct and incorrect predictions
correct = np.nonzero(y_pred == y_true)[0]
incorrect = np.nonzero(y_pred != y_true)[0]

print("Correct predicted classes:", correct.shape[0])
print("Incorrect predicted classes:", incorrect.shape[0])

accuracy = correct.shape[0] / (correct.shape[0] + incorrect.shape[0])
print("Accuracy on test data:", accuracy * 100)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Correct predicted classes: 99
Incorrect predicted classes: 1
Accuracy on test data: 99.0
